In [ ]:
import sys

In [ ]:
sys.path.insert(0,"../python/")
from search_utils import tweet_time_2_epoch

In [ ]:
from pymongo import MongoClient
import networkx as nx
import pandas as pd

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

# Utils

In [ ]:
def get_coll(coll_name,mongo_port=27017,mongo_db="twitter-crawler"):
    client = MongoClient('mongodb://localhost:%i/' % mongo_port)
    db = client[mongo_db]
    return db[coll_name], db

def find_some_docs(coll,sort_params=[('id',-1)],limit=10):
    res = coll.find().sort(sort_params).limit(limit)
    for item in res:
        print(item["id"],item["created_at"])

In [ ]:
def show_frequent_items(df,col,k=10):
    val_counts = pd.DataFrame(df[col].value_counts()[:k])
    frequent_users = [user_names[u_id] for u_id in val_counts.index]
    res_df = pd.DataFrame()
    res_df["id"] = val_counts.index
    res_df["name"] = frequent_users
    res_df["count"] = val_counts.values
    return res_df

In [ ]:
def get_graph_stats(df):
    edges = list(zip(df["src"],df["trg"]))
    G = nx.DiGraph()
    G.add_edges_from(edges)
    N = G.number_of_nodes()
    M = G.number_of_edges()
    wc_comp = nx.number_weakly_connected_components(G)
    sc_comp = nx.number_strongly_connected_components(G)
    return (N,M,wc_comp,sc_comp)

# Get collections

In [ ]:
raw_coll, db = get_coll("raw")
raw_begin_coll, _ = get_coll("raw_begining")

#### Number of documents in the collections

In [ ]:
raw_size = db.command("collstats", "raw")["count"]
raw_begin_size = db.command("collstats", "raw_begining")["count"]

In [ ]:
print("Number of docs in 'raw': %i" % raw_size)
print("Number of docs in 'raw_begining': %i" % raw_begin_size)

# Sample event

In [ ]:
doc = raw_coll.find_one()

doc

# Upper and lower bound of the available dataset

In [ ]:
find_some_docs(raw_begin_coll,sort_params=[("id",1)],limit=3)

In [ ]:
find_some_docs(raw_begin_coll,limit=3)

In [ ]:
find_some_docs(raw_coll,sort_params=[("id",1)],limit=3)

In [ ]:
find_some_docs(raw_coll,limit=3)

# Extract mention networks

In [ ]:
def get_mentions(coll,limit=None,use_only_tweets=True):
    res = coll.find().limit(limit) if limit != None else coll.find()
    num_tweets, num_retweets = 0, 0
    users = {}
    edges = []
    for item in res:
        if use_only_tweets and "RT " == item['text'][:3]:
            num_retweets += 1
            continue
        num_tweets += 1
        src_id, epoch = item['user']['id_str'], int(tweet_time_2_epoch(item['created_at']))
        users[src_id] = item['user']['name']
        if 'user_mentions' in item['entities']:
            for mention in item['entities']['user_mentions']:
                trg_id = mention['id_str']
                users[trg_id] = mention['name']
                msg = item["text"]
                edges.append((epoch,src_id,trg_id,msg))
    return edges, users, num_tweets, num_retweets

## a.) Extract mentions from multiple collections

In [ ]:
mentions_begin, user_names_begin, num_tweets_begin, num_retweets_begin = get_mentions(raw_begin_coll)

In [ ]:
mentions_df_begin = pd.DataFrame(mentions_begin,columns=["epoch","src","trg","text"])
print(len(mentions_df_begin))

In [ ]:
mentions_raw, user_names_raw, num_tweets_raw, num_retweets_raw = get_mentions(raw_coll)

In [ ]:
mentions_df_raw = pd.DataFrame(mentions_raw,columns=["epoch","src","trg","text"])
print(len(mentions_df_raw))

## b.) Rolland Garros mention network (concatenated)
   * first part: May 23 16:51 -> May 31
   * second part: June 01 -> WIP

In [ ]:
mentions_df = pd.concat([mentions_df_begin, mentions_df_raw])
mentions_df = mentions_df.reset_index(drop=True)
user_names_begin.update(user_names_raw)
user_names = user_names_begin
num_tweets = num_tweets_begin + num_tweets_raw
num_retweets = num_retweets_begin + num_retweets_raw

In [ ]:
mentions_df["src_str"] =  mentions_df["src"].apply(lambda x: user_names[x])
mentions_df["trg_str"] =  mentions_df["trg"].apply(lambda x: user_names[x])

### 1. Number of mentions

In [ ]:
print("Number of tweets: %i" % num_tweets)
print("Number of retweets: %i" % num_retweets)
print("Number of mentions extracted from tweets: %i" % len(mentions_df))

#### Export mentions to .csv

In [ ]:
cols_to_export = ["epoch","src","trg","src_str","trg_str"]
mentions_df.to_csv("/mnt/idms/fberes/network/roland_garros/data/rg17_mentions.csv",columns=cols_to_export,sep="|",index=False)

### 2. Popular source nodes

In [ ]:
show_frequent_items(mentions_df,"src",k=20)

### 3. Popular target nodes

In [ ]:
show_frequent_items(mentions_df,"trg",k=20)

### 4. Event distribution in time

In [ ]:
import time

In [ ]:
mentions_df["date"] = mentions_df["epoch"].apply(lambda x: time.strftime('%Y-%m-%d', time.localtime(x)))
mentions_df["time"] = mentions_df["epoch"].apply(lambda x: time.strftime('%H:%M:%S', time.localtime(x)))

In [ ]:
mentions_df.head()

In [ ]:
mentions_df["epoch"].hist(bins=100)

In [ ]:
num_of_mentions_by_day = mentions_df["date"].value_counts().sort_index()

In [ ]:
plt.figure()
plt.plot(num_of_mentions_by_day.values)
plt.xticks(range(len(num_of_mentions_by_day)),num_of_mentions_by_day.index,rotation='vertical')
plt.show()

### 5. Graph informations

In [ ]:
date_splits = sorted(list(mentions_df["date"].unique()))
#date_splits

In [ ]:
graph_stats = [] 
for split in date_splits[1:]:
    partial_df = mentions_df[mentions_df["date"] < split]
    graph_stats += [get_graph_stats(partial_df)]
graph_stats_df = pd.DataFrame(graph_stats, columns=["nodes","edges","weak_components","strong_components"])

In [ ]:
graph_stats_df

In [ ]:
plt.subplots(figsize=(16,5))

plt.subplot(1,2,1)
plt.title('Aggregated mention graph size')
plt.plot(graph_stats_df["nodes"],label="nodes")
plt.plot(graph_stats_df["edges"],label="edges")
plt.xticks(range(len(graph_stats_df)),date_splits[:-1],rotation='vertical')
plt.legend()

plt.subplot(1,2,2)
plt.title('Number of weak components in the aggregated mention graph')
plt.plot(graph_stats_df["weak_components"],label="number of ",c="r")
plt.xticks(range(len(graph_stats_df)),date_splits[:-1],rotation='vertical')
plt.show()